# IL2233 Lab 2 - ARIMA Model and Prediction

## Imports

In [ ]:
import matplotlib.pyplot as plt
import time
import statsmodels.api as sm
import numpy as np
from random import gauss, seed, randint
from pandas import Series, plotting, read_excel, DataFrame
from pandas.plotting import autocorrelation_plot
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.stattools import adfuller, acovf
from statsmodels.tsa.seasonal import seasonal_decompose, STL
from statsmodels.tsa.arima_process import arma_generate_sample, ArmaProcess
from statsmodels.tsa.ar_model import AutoReg, ar_select_order
from numpy.fft import fft, rfft
from scipy.io import loadmat
from scipy.signal import spectrogram
from math import sqrt

**Helper functions**

In [ ]:
# Line plot, histogram, density plot, box plot, lag-1 plot, ACF and PACF graphs
def plot_all(series: Series, lags: int):
    fig, axes = plt.subplots(nrows=4, ncols=2, figsize=(25, 25))
    fig.subplots_adjust()

    series.plot(ax=axes[0,0], kind='line', title='Line plot', xlabel='SV #', ylabel='Value')
    series.plot(ax=axes[0,1], kind='hist', title='Histogram', xlabel='Value', ylabel='frequency')
    series.plot(ax=axes[1,0], kind='box', title='Box plot')
    series.plot(ax=axes[1,1], kind='density', title='Density plot', xlabel='', ylabel='density')
    plotting.lag_plot(series, ax=axes[2,0], lag=1)
    plotting.lag_plot(series, ax=axes[2,1], lag=2)
    autocorrelation_plot(series, ax=axes[3,0])
    plot_pacf(series, lags=lags, ax=axes[3,1]);

# # Histogram, density plot, box plot
# def plot_hist_dens_box(series: Series):
#     fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(20, 10))
#     fig.subplots_adjust()

#     series.plot(ax=axes[0,0], kind='hist', title='Histogram', xlabel='Value', ylabel='frequency')
#     series.plot(ax=axes[0,1], kind='box', title='Box plot')
#     series.plot(ax=axes[1,0], kind='density', title='Density plot', xlabel='', ylabel='density')
#     plotting.lag_plot(series, ax=axes[2,0])

def ljungbox(series: Series):
    return sm.stats.acorr_ljungbox(series)

def adfuller_test(series: Series):
    result = adfuller(series, autolag='AIC')
    print(f'ADF Statistic: {result[0]}')
    print(f'p-value: {result[1]}')
    for key, value in result[4].items():
        print('Critial Values:')
        print(f'{key}, {value}')


def gen_arma(lagcoeffs_ar: list[int] = [], lagcoeffs_ma: list[int] = [], nsample: int = 250):
    lagcoeffs_ar = np.r_[1, -np.array(lagcoeffs_ar)] if len(lagcoeffs_ar) > 0 else np.array([1])
    lagcoeffs_ma = np.r_[1, np.array(lagcoeffs_ma)] if len(lagcoeffs_ma) > 0 else np.array([1])

    proc = ArmaProcess(lagcoeffs_ar, lagcoeffs_ma)

    generated_time_series = proc.generate_sample(nsample)
    return generated_time_series, proc

# Task 1: Stationarity of AR models

In [ ]:
s1, p1 = gen_arma(lagcoeffs_ar=[0.8])
s2, p2 = gen_arma(lagcoeffs_ar=[-1.1])
s3, p3 = gen_arma(lagcoeffs_ar=[1, -0.5])
s4, p4 = gen_arma(lagcoeffs_ar=[1, 0.5])

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(20, 10))
fig.subplots_adjust()
Series(s1).plot(ax=axes[0,0], kind='line', title='1)')
Series(s2).plot(ax=axes[0,1], kind='line', title='2)')
Series(s3).plot(ax=axes[1,0], kind='line', title='3)')
Series(s4).plot(ax=axes[1,1], kind='line', title='4)')

**Judging visually:**

None of the time series seem to be weakly stationary. In the case of 2), the mean may be constant but the variance is not.
We find that 3) could be potentially stationary, but further tests are required to determine this.

**Judging by coefficients:**

To determine if the generated time series are stationary we can check the following constraints on the coefficients:
- For p = 1: <u>−1 < $\phi_1$ < 1</u>.
- For p = 2: <u>−1 < $\phi_2$ < 1</u> AND <u>$\phi_2$ + $\phi_1$ < 1</u> AND <u>$\phi_2$ − $\phi_1$ < 1</u>.

- 1) AR(1) = $0.8x_{t-1} + \epsilon_t           \rightarrow -1 < \phi_1 = 0.8 < 1$, which **fulfills** the stationarity condition for $p = 1$
- 2) AR(1) = $-1.1x_{t-1} + \epsilon_t          \rightarrow -1 > \phi_1 = -1.1$, which does **not fulfill** the stationarity condition for $p = 1$
- 3) AR(2) = $1x_{t-1} - 0.5x_{t-2} \epsilon_t  \rightarrow -1 < \phi_2 = -0.5 < 1$, $\phi_2 + \phi_1 = 0.5 < 1$, and $\phi_2 - \phi_1 = 0.5 < 1$, which **fulfills** the stationarity condition for $p = 2$
- 4) AR(2) = $1x_{t-1} + 0.5x_{t-2} \epsilon_t  \rightarrow -1 < \phi_2 = 0.5 < 1$, $\phi_2 + \phi_1 = 1.5 > 1$, and $\phi_2 - \phi_1 = -0.5 < 1$, which does **not fulfill** the stationarity condition as the second criteria is not fulfilled

**Built-in function in statsmodels:**

From the output below, we validate that manual stationarity calculations were correct.

In [ ]:
print(f"AR process 1 is stationary: {p1.isstationary}")
print(f"AR process 2 is stationary: {p2.isstationary}")
print(f"AR process 3 is stationary: {p3.isstationary}")
print(f"AR process 4 is stationary: {p4.isstationary}")

**Augmented Dickey-Fuller Test:**

One final stationarity test is done on the time series applying the ADF test. For each of the test outputs, we check if the ADF-statistic is less than all the critical values and also if the p-value is less than 0.5.

Our findings indicate the same results as previous tests.

In [ ]:
adfuller_test(Series(s1))
print("-"*40)
adfuller_test(Series(s2))
print("-"*40)
adfuller_test(Series(s3))
print("-"*40)
adfuller_test(Series(s4))

## Task 1 - Questions

- **With visual inspection, how do you identify if a time series is stationary or not?**

By visual inspection one can check what the mean seems to be for various intervals of the time series, but also for the time series as a whole. Variance is also easy to visually inspect over slices in time and for the whole series. Another easy inspection methods includes finding seasonality, i.e. repeating patterns.
'
- **How do you judge the stationarity of time series using the unit-root method? Does it always give correct results?**

We use the ADF test to judge stationarity. Based on the ADF statistic, negative values implies less probability of non stationarity.

- **What is the role of component $\epsilon_t$ in the model? Why is it important?**

$\epsilon_t$ is a Gaussian white noise with normal distribution (0, 1). There is always a measure of error to future values based on prediction from past data points, which need to be taken into consideration. The error $\epsilon_t$ does this.

- **To have an AR(p) model be stationary, is there any requirement on the auto-regressive coefficients? List the constraints for AR(1) and AR(2) models**

The following constraints are put on the coefficients to determine stationarity of the AR(p) model, where $\phi_t$ is the coefficient:

- For p = 1: <u>−1 < $\phi_1$ < 1</u>
- For p = 2: <u>−1 < $\phi_2$ < 1</u> AND <u>$\phi_2$ + $\phi_1$ < 1</u> AND <u>$\phi_2$ − $\phi_1$ < 1</u>

# Task 2: ACF, PACF of AR models

### Are all of them stationary? **Judging visually:**
At a first glance, 2) and 4) seem to be weakly stationary. 1) show signs of great variation of mean and variance at different intervals, and thus can't be considered weakly stationary. 3) could potentially be weakly stationary, but further tests are required to determine this.

### Are there any outliers? Why?
Looking at the box plot, we can see outliers in most cases. This, of course, depends on the gaussian white noise whether outliers exist or not. Our theory is that when the data points in a time series differ by a certain variance from the mean, this spawns outliers either above or below the box plot (depending on the data point being above or below the mean)

### Do you observe any auto-correlation from the lag plots?
The lag-1 plot seems to give a measure of autocorrelation, at least more than lag-2. Despite this, there is no clear autocorrelation for any of the given models.

### What characteristics can you observe from the ACF graphs of the AR(p) models?
Over time, the ACF graph converges to remain inside the confidence interval.

### What characteristics can you observe from the PACF graphs of the AR(p) models?
After the relevant lag = p + 1, the data points will be inside the confidence interval. This means that there is no considerable autocorrelation after the p + 1 lag, and we believe this is the case beceause the AR model does not use any further "lag" than that.

In [ ]:
s1, p1 = gen_arma(lagcoeffs_ar=[0.8], nsample=1000)
s2, p2 = gen_arma(lagcoeffs_ar=[-0.8], nsample=1000)
s3, p3 = gen_arma(lagcoeffs_ar=[1, -0.5], nsample=1000)
s4, p4 = gen_arma(lagcoeffs_ar=[-1, -0.5], nsample=1000)

plot_all(Series(s1), lags=40);
plot_all(Series(s2), lags=40);
plot_all(Series(s3), lags=40);
plot_all(Series(s4), lags=40);

# Task 3: Invertibility, ACF, PACF of MA models

### Are all of them invertible, just by visual inspection?
It is uncertain whether invertibility can be determined through visual inspection. 

### Judge whether each time series is invertible or not, by its models’s auto-regressive coefficient values.
The following constraints are put on the coefficients to determine invertability of the MA(p) model, where $\theta_t$ is the coefficient:

- For q = 1: <u>−1 < $\theta_1$ < 1</u>
- For q = 2: <u>−1 < $\theta_2$ < 1</u> AND <u>$\theta_2$ + $\theta_1$ < 1</u> AND <u>$\theta_1$ − $\theta_2$ < 1</u>

- 1) MA(1) = $\epsilon_t - 2 \epsilon_{t-1} \rightarrow -1 < \theta_1 = 2 < 1$, which **not fulfill** the invertibility condition for $p = 1$.
- 2) MA(1) = $\epsilon_t - 0.5 \epsilon_{t-1} \rightarrow -1 > \theta_1 = 0.5$, which does **fulfill** the invertibility condition for $p = 1$.
- 3) MA(2) = $\epsilon_t - \frac{4}{5} \epsilon_{t-1} + \frac{16}{25} \epsilon_{t-2} \rightarrow -1 < \theta_2 = 16/25 < 1$, $\theta_2 + \theta_1 = -0.16 > -1$, and $\theta_1 - \theta_2 = 0.16 < 1$, which **fulfills** the invertibility condition for $p = 2$.
- 4) MA(2) = $\epsilon_t - \frac{5}{4} \epsilon_{t-1} + \frac{25}{16} \epsilon_{t-2} \rightarrow -1 < \theta_2 = 25/16$, but $> 1$ which does **not fulfill** the invertibility because of the first criterion for p = 2.

### Call the following statsmodel function to judge if each MA process is invertible
The function calls show us that processes 1 and 4 are **not** invertible, whereas processses 2 and 3 **are** invertible.

### Draw histogram, density plot, and box plot for each time series with 1000 data points. Are there any outliers? Why?
Outliers occur in most cases. They exist because the error distribution can sometimes generate values outside the standard confidence intervals simply and then they become outliers in the box plot.

### Draw lag-1 and lag-2 plots for each time series. Do you observe any auto-correlation from the lag plots?
There is no discernible auto-correlation in the lag plots.

## Questions

### Are all the MA models invertible? If not, which ones are invertible and which ones are not invertible?
The second (2nd) and third (3rd) are invertble, the remaining are not.

### What characteristics can you observe from the ACF graphs of the MA(q) models?
In regards to invertibility, since two different MA(q) models can generate the same ACF graph, not conclusion can be drawn.

### What characteristics can you observe from the PACF graphs of the MA(q) models?
In regards to invertibility, since two different MA(q) models can generate the same PACF graph, not conclusion can be drawn.


### To have an MA(q) model be invertible, is there any requirement on the auto-regressive coefficients? List the constraints for MA(1) and MA(2) models.
The following constraints are required on the MA(1) and MA(2) models for invertibility

- For q = 1: <u>−1 < $\theta_1$ < 1</u>
- For q = 2: <u>−1 < $\theta_2$ < 1</u> AND <u>$\theta_2$ + $\theta_1$ < 1</u> AND <u>$\theta_2$ − $\theta_1$ < 1</u>

In [ ]:
s1, p1 = gen_arma(lagcoeffs_ma=[-2], nsample=1000)
s2, p2 = gen_arma(lagcoeffs_ma=[-0.5], nsample=1000)
s3, p3 = gen_arma(lagcoeffs_ma=[-4/5, 16/25], nsample=1000)
s4, p4 = gen_arma(lagcoeffs_ma=[-5/4, 25/16], nsample=1000)

plot_all(Series(s1), lags=40);
plot_all(Series(s2), lags=40);
plot_all(Series(s3), lags=40);
plot_all(Series(s4), lags=40);

print("Is process 1 invertible?", p1.isinvertible)
print("Is process 2 invertible?", p2.isinvertible)
print("Is process 3 invertible?", p3.isinvertible)
print("Is process 4 invertible?", p4.isinvertible)

# Task 4 - Stationarity, ACF and PACF of ARMA models